In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import re
import emoji
import jieba
import jieba.posseg as pseg #这个包可以标注词性，我们只需要n,v,a
from zhconv import convert

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
pd.read_csv("data_zhihu_addition_ChatGPT_filtered.csv")

,source,comments,time,tokenization,tokenization_filtered
0,zhihu,是啊，为什么没有做出来呢？在刚出现的时候，我也想过这个问题。像题主所说的，百度在技术、语言处...,2022-12-27 00:12:57,像题 所说 百度 技术 语言 技术 研发 能力 依托 技术 语言 技术 诞生 产品 公司 做...,百度 技术 语言 技术 能力 依托 技术 语言 技术 诞生 产品 公司 做出 产品 百度 能...
1,zhihu,百度并不是没有做，文心系列其实也是学习使用同样的思路，不过效果肯定还是跟不上。说百度没志气光...,2022-12-20 00:44:51,百度 文心 系列 学习 思路 效果 肯定 跟不上 百度 没志气 广告 肯定 感觉 百度 技术...,百度 文心 学习 思路 效果 肯定 跟不上 百度 没志气 广告 肯定 感觉 百度 技术 氛围...
2,zhihu,为什么谷歌没有做出这种产品？,2022-12-27 09:36:36,谷歌 做出 产品,做出 产品
3,zhihu,之前在 组实习过。个人认为百度在一众大厂里还是很看重技术研发的，但 这种不确定性极高的项目很...,2022-12-20 01:59:18,实习 百度 一众 厂里 看重 技术 研发 不确定性 极高 项目 很难 感觉 ...,实习 百度 厂里 看重 技术 不确定性 项目 感觉 成功 先例 资源 提不上 能力 有限 基...
4,zhihu,原因是多方面的，都是个人浅显的理解。一直是在主攻，其他公司大部分都在做方向，、出来的时候大家...,2022-12-22 21:02:08,原因 多方面 浅显 理解 主攻 公司 大部分 方向 看不上 发现 好像 真给 出点 东西 搜...,原因 浅显 理解 公司 方向 看不上 发现 好像 出点 搜索引擎 百度 搜索引擎 公司 投入...
...,...,...,...,...,...
3011,zhihu,中文网络上的八股文太多，让抢了饭碗，有人急了。,2023-01-31 12:28:11,中文 网络 八股文 饭碗 有人,网络 饭碗
3012,zhihu,从刚推出时的简单聊天，到现在已经被开发出写诗、考试、甚至写论文的功能，未来发展前景广阔。机器...,2023-01-31 11:05:48,从刚 推出 简单 聊天 开发 写诗 考试 论文 功能 未来 发展前景 广阔 机器人 产业 国...,推出 简单 开发 写诗 功能 广阔 机器人 产业 国家 层面 政策 支持 老龄化 社会 劳动...
3013,zhihu,人工智能技术其实应用已经非常的广泛了，比如小区门口的人脸识别技术，比如医疗上的医疗机器人等等...,2023-01-31 11:05:44,人工智能 技术 小区 门口 人脸识别 技术 医疗 医疗 机器人 科技 技术 发展 社会 人工...,人工智能 技术 小区 人脸识别 技术 医疗 医疗 机器人 科技 技术 社会 人工智能 智能 ...
3014,zhihu,和我都能被指导成萌头黑猫泠仆风格，但我更胜几千筹。期待有人喂食我泠力，我会把不明所以的台词发...,2023-02-06 11:31:36,指导 成萌头 黑猫 风格 几千 期待 有人 喂食 我泠力 我会 不明 台词 发给你 作品 可...,指导 萌头 黑猫 风格 期待 喂食 我会 不明 台词 作品 可爱 低于 动漫 主角 世界 萌...


#### 单平台数据初步预处理

In [28]:
df_zhihu = pd.read_csv('data_zhihu_addition_ChatGPT.csv')
df_zhihu['source'] = 'zhihu'
print(f'Zhihu Addition Set Shape = {df_zhihu.shape}')  
print('Zhihu Addition Set Memory Usage = {:.2f} MB'.format(df_zhihu.memory_usage().sum() / 1024**2))

Zhihu Addition Set Shape = (3199, 9)
Zhihu Addition Set Memory Usage = 0.22 MB


In [29]:
df_zhihu.head()

,author,fans_count,content,created_time,updated_time,comment_count,voteup_count,url,source
0,科技鸣人,274995,是啊，为什么没有做出来呢？在chatGPT刚出现的时候，我也想过这个问题。像题主所说的，百度...,2022-12-27 00:12:57,2022-12-27 00:12:57,4,124,https://www.zhihu.com/api/v4//answers/2817371412,zhihu
1,Phoenix Cat,6999,百度并不是没有做，文心系列其实也是学习gpt使用同样的思路，不过效果肯定还是跟不上。说百度没...,2022-12-20 00:44:12,2022-12-20 00:44:51,0,13,https://www.zhihu.com/api/v4//answers/2808352416,zhihu
2,皮蛋XXXXL,417,为什么谷歌没有做出chatGPT这种产品？,2022-12-27 09:36:36,2022-12-27 09:36:36,0,2,https://www.zhihu.com/api/v4//answers/2817621817,zhihu
3,知乎用户,3193,之前在 PLATO 组实习过。个人认为百度在一众大厂里还是很看重技术研发的，但 ChatGP...,2022-12-20 01:59:18,2022-12-20 01:59:18,63,110,https://www.zhihu.com/api/v4//answers/2808386208,zhihu
4,匿名用户,0,原因是多方面的，都是个人浅显的理解。GPT一直是openai在主攻，其他公司大部分都在做be...,2022-12-20 00:18:46,2022-12-22 21:02:08,0,28,https://www.zhihu.com/api/v4//answers/2808336053,zhihu


In [30]:
# rename columns
df_zhihu = df_zhihu.rename(columns={'content':'comments','updated_time':'time'})

# convert data type
df_zhihu['time'] = df_zhihu['time'].astype('datetime64[ns]')

# extract data(source, topics, comments, time)
df_zhihu = df_zhihu[['source','comments','time']]

In [31]:
# merge data from four platform
df = df_zhihu.copy()

#### 规范化预处理（Cleansing）

In [32]:
# drop duplicated
print('Data size = ',df.shape)
df = df.drop_duplicates(subset='comments')
# print('Duplicates data size = ',df.duplicated(subset='comments').sum())
print('After drop duplicated data size = ',df.shape)

# drop data missing time or comments
df.dropna(subset=['comments'],inplace=True)
df.dropna(subset=['time'],inplace=True)
print('After drop missing data size = ',df.shape)
df.index = range(0,len(df))

Data size =  (3199, 3)
After drop duplicated data size =  (3032, 3)
After drop missing data size =  (3031, 3)


In [33]:
# data clean
def clean(text):
    '''
    正则化
    '''
    
    # 清除@和回复/转发中的用户名
    text = re.sub(r"(回复)?(//)?\s*@\S*?\s*(:| |$)", "", text).strip()
    
    # 清除emoji表情
    text = emoji.demojize(text).strip()   
    
    # 清除##话题内容
    text = re.sub(r"#\S+#", "", text).strip()    
    
    # 清除网址link
    URL_REGEX1 = re.compile('</?\w+[^>]*>')
    URL_REGEX2 = re.compile('(https|http|ftp|file)://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]')
    text = re.sub(URL_REGEX1, "", text).strip()
    text = re.sub(URL_REGEX2, "", text).strip()
    
    # 清除数字
    text = re.sub('[\d]+','',text).strip()
    
    # 清除字母
    text = re.sub('[a-zA-Z]+','',text).strip()
    
    # 清除无意义的词语
    text = text.replace("转发微博", "").strip()
    
    # 合并多余的空格
    text = re.sub(r"\s+", " ", text).strip() 
    
    # 清除标点
    r = "[_.+-=——$%^~@#￥%……&*《》<>「」{}【】()“”\"/]+"
    r_cn = "[\\u3002\\uff1b\\uff0c\\uff1a\\u201c\\u201d\\uff08\\uff09\\u3001\\uff1f\\u300a\\u300b]"
    text = re.sub(r, '', text).strip()
#     text = re.sub(r_cn, '', text).strip()

    
    return text

In [34]:
def simplify(text):
    try:
        return convert(text,'zh-hans')
    except:
        return None

In [35]:
tqdm.pandas()
df['comments'] = df['comments'].progress_apply(clean) # 综合正则化处理
df['comments'] = df['comments'].progress_apply(simplify) # 繁体字简体化
# df['topics'] = df['topics'].progress_apply(clean) # 繁体字简体化

100%|█████████████████████████████████████| 3031/3031 [00:00<00:00, 9603.29it/s]


In [36]:
# drop duplicated
print('Data size = ',df.shape)
df.drop_duplicates(subset='comments',inplace=True)
print('After drop duplicated data size = ',df.shape)

Data size =  (3031, 3)
After drop duplicated data size =  (3016, 3)


In [37]:
df.tail(50)

,source,comments,time
2981,zhihu,试了一下党建文章能不能用 发现不行,2023-01-31 15:42:45
2982,zhihu,高考第一肯定是。后面绝大部分艺术也可能被超越了。普通老百姓想那么多干啥，搞的跟自己是靠艺术家...,2023-01-31 15:35:44
2983,zhihu,首先淘汰缺乏独立思考能力的唐飞们,2023-01-31 15:13:06
2984,zhihu,懂什么是藏头诗吗？懂什么是信息价吗？差了数量级呀 。。。。,2023-01-31 15:10:43
2985,zhihu,上午花了点小钱，去自己试验了一下。简单小结就是，如果你自己有东西，找它问几句，或者写一个，再...,2023-01-31 15:10:33
2986,zhihu,这就用它来回答知乎回答。,2023-01-31 15:07:53
2987,zhihu,就是解放生产力而已。未来即将进入真正的自媒体时代，网剧泛滥，程度可以参考网文。可以参考一下西...,2023-01-31 15:02:42
2988,zhihu,有些学生用它来写作业，帮自己生成答案，生成论文。这些学生是解放了自己，还是被支配了呢？,2023-01-31 14:56:04
2989,zhihu,如果是免费的可以一试，收费就打扰啦。,2023-01-31 14:51:40
2990,zhihu,人工智能只是一个辅助工具，没有自主意识，就不存在支配地位。肯定是生产力，至少，它比搜索引擎来...,2023-01-31 14:37:10


#### 数据过滤预处理（overview_data）

In [38]:
def detect_encoding(file):
    #获取文件最可能的编码格式
    import chardet
    with open(file, "rb") as f:
        r = f.read()
    e = chardet.detect(r)
    encoding = e.get("encoding")
    return encoding

def get_stopwords(stop_words_file, encoding):
    """
    读取停用词文件
    参数：
        停用词文件
    返回：
        停用词列表
    """
    
    with open(stop_words_file,encoding = encoding) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

def get_tokenization(text):
    """
    使用jieba分词并剔除停用词
    参数：
        文本字符串
    返回：
        以空格连接的词串
    """
    word_token = []
    for word in jieba.lcut(text.strip(), cut_all=False): #精确模式
        if word not in STOPWORDS:
            word_token.append(word)
    return ' '.join(word_token)

def clear_singlechar(text):
    """
    清除切词后的单个字符
    """
    word_cleared = []
    for word in text.strip().split(r' '):
        if len(word)!=1:
            word_cleared.append(word)
    return ' '.join(word_cleared)

def pos_tag_filter_nva(text):
    finance_noisy = ["疫情", "股市", "风险投资", "风险融资", "风险资本", "信贷风险", "贷款风险", "投资风险", "交易风险", "信用风险", "风险评分", "金融风险", "基金", "个股"]
    
    words = pseg.cut(text)
    filtered_words = []
    for word, flag in words:
        # a = ("{0}/{1}".format(word, flag))
        if (flag == 'n'or flag == 'v' or flag == 'a') and (len(word) > 1) and word not in finance_noisy:
            filtered_words.append(word)
    return ' '.join(filtered_words)

In [39]:
# 停用词表
stop_qua = r'ー、一、二、三、四、五、一点、两个、三个、一些、一种、几个、几种；和、跟、与、既、同、及、而、况、况且、何况、乃至、则、乃、就、而、便、于是、然后、至于、说到、此外、像、如、一般、比方、接着、却、虽然、但是、然而、而、偏偏、只是、不过、至于、致、不料、岂知、原来、因为、由于、以便、因此、所以、是故、以致、或、或者、还是、抑、非…即、不是…就是、若、如果、若是、假如、只要、除非、假使、倘若、即使、假若、要是、譬如、像、好比、如同、似乎、等于；不如、不及、与其…不如、若…则、虽然…可是、虽然、固然、尽管、纵然、即使、不但、不仅、而且、何况、并、且、不管、只要、除非、以、以便、以免、为了'

STOPWORDS = list(set(get_stopwords('../stopwordsHIT.txt', encoding=detect_encoding('../stopwordsHIT.txt')) + get_stopwords(r'../stopwords.txt', encoding='gbk') + stop_qua.split(r'、')))
print(f'Number of stop words = {len(STOPWORDS)}')

Number of stop words = 2576


In [40]:
# 切词
df = df.copy()
jieba.load_userdict('../dict.txt')

tqdm.pandas()
df['tokenization'] = df['comments'].progress_apply(get_tokenization)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/29/27cgqwrx7_18wcd66rpffzlc0000gn/T/jieba.cache
Loading model cost 0.360 seconds.
Prefix dict has been built successfully.
100%|██████████████████████████████████████| 3016/3016 [00:13<00:00, 228.08it/s]


In [41]:
# 清除单个字符
df = df.copy()
tqdm.pandas()
df['tokenization'] = df['tokenization'].progress_apply(clear_singlechar)

100%|████████████████████████████████████| 3016/3016 [00:00<00:00, 94977.26it/s]


In [42]:
# 过滤切词结果，保留n/v/adj
df = df.copy()
jieba.load_userdict('../dict.txt')

tqdm.pandas()
df['tokenization_filtered'] = df['tokenization'].progress_apply(pos_tag_filter_nva)

FileNotFoundError: [Errno 2] No such file or directory: '../ITP/Data collection/dict.txt'

#### 对微博含有广告、打榜等重复噪音数据的补充处理

In [ ]:
# # 1.根据关键词对微博数据再做初筛 (主要针对微博数据)
# words_list = ['风险','安全隐患','威胁','危险','危机','后果','危害','安全事件','预防','安全事故','风险管理','负面影响','灾害','困境','欺诈','失误','漏洞']
# risk_words = str()
# for item in words_list:
#     risk_words = risk_words+str(item)+'|'
    
# # 筛掉所有不含有“人工智能”和“risk_words”中关键词的内容
# data_weibo_filtered = data_weibo[(data_weibo['tokenization_filtered'].str.contains("人工智能"))==True]

# # 筛掉所有不含有“人工智能”和“words_list”中关键词的内容
# data_weibo_filtered= data_weibo_filtered[(data_weibo_filtered['tokenization_filtered'].str.contains(risk_words))==True]

In [ ]:
# # 2.使用fuzzywuzzy库，对数据进行遍历，剔除掉相似度75%以上的数据（主要针对微博数据）

# # Final Version (itertools.islice)
# def Similarity(index_tuples_list):
#     res = []
# #     print(index_tuples_list)
#     for p in index_tuples_list:
#         similarity = fuzz.ratio(data_weibo_filtered['comments'][p[0]],data_weibo_filtered['comments'][p[1]])
#         if similarity > 75:
#             res.append([p[0],p[1],similarity])
#     return res

# pair_similarity = []
# begin_all = time.time()

# obj_list = []
# executor = ThreadPoolExecutor(max_workers=8)

# # 分批次 1000000000*18
# for i in tqdm(range(3000000000,4000000000,5000000),position=0, leave=True):
#     obj_list.append(itertools.islice(itertools.combinations(data_weibo_filtered.index.to_list(),2),i,i+5000000)) 

# for res in executor.map(Similarity, obj_list):
#     if (len(res)) > 0:
#         pair_similarity.extend(res)
    
# end_all = time.time()
# print(end_all-begin_all)

In [ ]:
# 3.删除分词过滤后无词的数据
print(df.shape)
df.dropna(subset=['tokenization_filtered'], inplace=True)
# df.reset_index(inplace=True, drop=True)
print(df.shape)

In [ ]:
# 4.删除分词+过滤后词数<5的数据(可选操作)
'''
item_word_below_5 = []

tqdm.pandas()
df['word_count'] = df['tokenization_filtered'].progress_apply(lambda x: len(x.split()))
df = df[df['word_count']>=5][['source','comments','time','tokenization','tokenization_filtered','word_count']]# ,'topics'
print(df.shape)
'''

In [101]:
df[['source','comments','time','tokenization','tokenization_filtered']].to_csv('data_zhihu_ChagtGPT_addition_filtered.csv', index=False, encoding='utf-8-sig')